## Preface

You are encouraged to play with the code, run it on your own data, and exchange knowledge with other participants.

If you don't have your own data, you can use the `tibia.tiff` provided. The data is of a CT scan of a mouse shin bone ("tibia"), and is courtesy of Mark Hopkinson (Royal Veterinary College). As part of the tutorial, you will convert this 3D image stack to a OME-zarr, a chunked pyramidal file format. The data is relatively small (~250 MB) - to make this tutorial run fast, but in "the wild" the techniques shown here are mostly useful for large data, that doesn't fit into memory.

## Reading the TIFF data

In a first step, we read in the TIFF data with `bioio` and its `tifffile` reader plugin - this will work well for simple and small TIFF files like the tibia scan, but it might fail for larger formats or folders of 2D tiffs - see the Advanced tab for solution strategies for this. Make sure to update the local file path to point to the right place.

<details>

<summary>Advanced: `bioio` readers and `dask`</summary>

* If you'd like to try this on proprietary data, check the [`bioio` website for an appropriate reader](https://bioio-devs.github.io/bioio/OVERVIEW.html#reader-installation).
* If you'd like to try this on large data, you will be [interested in `bioio.BioImage.get_image_dask_data`](https://bioio-devs.github.io/bioio/bioio.html#bioio.bio_image.BioImage.get_image_dask_data), which allows you to access [array data lazily through `dask`](https://docs.dask.org/en/stable/array.html).
</details>



In [ ]:
from pathlib import Path
from bioio import BioImage, plugin_feasibility_report

tiff_image_path = Path("/media/alessandro/T7/tibiae/acc52_10um.tif")
assert tiff_image_path.exists()
print(plugin_feasibility_report(tiff_image_path))

tiff_image = BioImage(tiff_image_path)

## Exploring the TIFF image

In the next section, we can have a look at some of the image properties

In [ ]:
print(tiff_image.dims)
print(tiff_image.metadata)

For the example data, note the three spatial dimensions, and that there is just one timepoint and channel, and that the order of the dimensions is TCZYX.
We also note that the metadata is dubious (this is common!): from the filename, we suspect that the pixel size is 10um, rather than 1.99 pixel.

We might also want to plot some slices through the 3D image, and we see some cross-sections of a long bone:

In [ ]:
import matplotlib.pyplot as plt

def plot_central_slice(image_data, axis=0):
    idx = image_data.shape[axis] // 2
    central_slice = image_data.take(indices=idx, axis=axis)
    plt.imshow(central_slice, cmap='gray')
    plt.title(f'Central slice along axis {axis}')
    plt.axis('off')
    plt.show()

# the BioImage is 5D and follow TCZYX convention
# we only want the spatial dimensions
plot_central_slice(tiff_image.data[0,0], axis=0)
plot_central_slice(tiff_image.data[0,0], axis=1)

## Converting to chunked format (zarr)

We have seen that we need to make decisions around storage location, chunk size and compression library and level to write chunked file formats. Let's see whether we can do each of these in Python below!

First we remove the extra time and channel dimensions, for simplicity

In [ ]:
import numpy as np
tiff_image_3d_data = np.squeeze(tiff_image.data)

Next, we specify the chunk size and the compression in an "array specification" (`ArraySpec`)

In [ ]:
from pydantic_zarr.v2 import ArraySpec
from numcodecs import Zstd

array_spec = ArraySpec(
    shape=tiff_image_3d_data.shape,
    dtype=tiff_image_3d_data.dtype,
    chunks=(64,64,64),
    compressor=Zstd(level=5),
)
print(array_spec)

Now, we specify where we want to story the chunked file - by default, we'll make a folder next to the tiff file.

In [ ]:
import zarr

zarr_path = tiff_image_path.parent / "chunked_tibia"
Path.mkdir(zarr_path, exist_ok=True)
print(f"Created a folder at {zarr_path}")
print(f"Folder contents before creating zarr store {list(zarr_path.iterdir())}")

store = zarr.DirectoryStore(zarr_path)
zarr_array = array_spec.to_zarr(store=store, path="/")
print(f"Folder contents after creating zarr store {list(zarr_path.iterdir())}")

Note that `zarr_array` doesn't contain any pixel data yet (it's full of zeros) - it now just know where and how it should store data.
So finally, we need to copy the data to the chunked array.

In [ ]:
zarr_array[:] = tiff_image_3d_data[:]

Let's see what's inside the zarr folder now:

In [ ]:
print(f"Folder contents after copying pixel data into zarr {list(zarr_path.iterdir())}")
print(f"There are {len(list(zarr_path.iterdir()))-2} data subfolders")

Can you explain the number of subfolders?

# Converting to pyramidal file format (OME-zarr)

Now we can work on adding lower levels of resolution to the array, and specifying metadata. First, we re-use the same array specification as before, and specity voxel size, units and image name. We also specify that the current array is level 0 of the pyramid.

In [ ]:
from ome_zarr_models.v04 import Image
from ome_zarr_models.v04.axes import Axis

voxel_size = 10
ome_zarr_image = Image.new(
    array_specs = [ArraySpec.from_array(zarr_array)],
    paths = ["level0"],
    axes = [
        Axis(name="z", type="space", unit="um"),
        Axis(name="y", type="space", unit="um"),
        Axis(name="x", type="space", unit="um")
    ],
    global_scale = [voxel_size, voxel_size, voxel_size],
    scales = [[1, 1, 1]],
    translations = [[0, 0, 0]],
    name = "mouse tibia",
)
print(ome_zarr_image)

Now we add a new storage location ("store") for the pyramidal file.

In [ ]:
ome_zarr_path = tiff_image_path.parent / "pyramidal_chunked_tibia"
ome_store = zarr.DirectoryStore(ome_zarr_path)
ome_group = ome_zarr_image.to_zarr(ome_store, path='', overwrite=True)
print(ome_group)

The code to access the array is quite complicated - we need to fill it with values again!

In [ ]:
level0_array = ome_group[ome_zarr_image.attributes.multiscales[0].datasets[0].path]
level0_array[:] = zarr_array[:]

Now let's create more levels by downsampling!

In [ ]:
import math

full_res_spec = ArraySpec.from_array(zarr_array)
print("Original array specification: ", full_res_spec)

downsample_levels = [0, 1, 2]
downsampled_specs = [
    full_res_spec.model_copy(
        update={"shape": tuple(math.ceil(i / 2**d) for i in full_res_spec.shape)
    }) for d in downsample_levels
]
print("Downsampled array specifications: ", downsampled_specs)

In [ ]:
ome_zarr_image = Image.new(
    array_specs = downsampled_specs,
    paths = [f"level{d}" for d in downsample_levels],
    axes = [
        Axis(name="x", type="space", unit="um"),
        Axis(name="y", type="space", unit="um"),
        Axis(name="z", type="space", unit="um")
    ],
    global_scale = [voxel_size, voxel_size, voxel_size],
    scales = [[2**d, 2**d, 2**d] for d in downsample_levels],
    translations = [[0, 0, 0] for d in downsample_levels],
    name = "mouse tibia"
)
print(ome_zarr_image)

ome_group = ome_zarr_image.to_zarr(ome_store, path='', overwrite=True)
print(ome_group)

In [ ]:
for d in downsample_levels:
    level_d_array = ome_group[ome_zarr_image.attributes.multiscales[0].datasets[d].path]
    level_d_array[:] = zarr_array[::2**d, ::2**d, ::2**d]

In [ ]:
import napari

viewer = napari.Viewer()
viewer.open(ome_zarr_path, plugin="napari-ome-zarr")
napari.run()


Some areas for further exploration:
* what parts of the code would you need to change if the data didn't fit in your memory
* how could you change the chunk size?

Or you can move on to the next tutorial, about reading and thresholding OME-zarr data!